In [46]:
import pandas as pd
import numpy as np
import random

import os


import pandas as pd
import os

# Đường dẫn các file dữ liệu
baseline_file = "~/GMM-nfst/Results/Baseline_noise73.csv"
model_files = {
    # "data_BoTIoT.csv": "~/GMM-nfst/Results/OURMODEL/SCALERS/K_73_old_data_BoTIoT.csv0.csv",
    # "data_CICIoT2023.csv": "~/GMM-nfst/Results/OURMODEL/SCALERS/K_73_old_data_CICIoT2023.csv0.csv",
    # "data_N_BaIoT.csv" : "~/GMM-nfst/Results/OURMODEL/SCALERS/K_73_old_data_N_BaIoT.csv0.csv",
    "data_ToNIoT.csv": "~/GMM-nfst/Results/OURMODEL/SCALERS/K_73_old_data_ToNIoT.csv0.csv"
}

scaler_names = ['MinMaxScaler', 'Normalizer', 'StandardScaler', 'QuantileTransformer', 'RobustScaler']
metric_names = ['AUCROC'] # , 'AUCPR', 'Accuracy', 'MCC', 'F1 Score']   
# Đọc dữ liệu baseline
baseline_df = pd.read_csv(os.path.expanduser(baseline_file))

# Lặp qua từng mức noise
for noise in [0, 1, 3, 5]: 
    baseline_noise = baseline_df[baseline_df["noise_percentage"] == noise]
    baseline_noise = baseline_noise[baseline_noise['Model'] != 'LUNAR']

    for dataset, model_file_path in model_files.items():
        

        # Đọc dữ liệu mô hình của bạn
        model_df = pd.read_csv(os.path.expanduser(model_file_path))
        model_noise = model_df[model_df["noise_percentage"] == noise]

        for metric in metric_names: 

            best_metric = -1
        
            # Tìm mô hình của bạn có AUCROC cao nhất
            for scaler in scaler_names:
                model_scaler = model_noise[model_noise["scaler"] == scaler]
                for _, row in model_scaler.iterrows():
                    if row[metric] > best_metric:
                        best_metric = row[metric]
    
            best_model_row = None
            
            # if best_model_row is None:
            #     best_model_row = pd.Series({'Model': 'ourmodel', dataset: 0})
            # else:

            col_name = f"{dataset}_{metric}"
            best_model_row = pd.Series({'Model': 'ourmodel', col_name: best_metric})

            
            # Lọc baseline theo dataset
            dataset_baseline = baseline_noise[baseline_noise["Dataset"] == dataset]
            selected_columns = dataset_baseline[['Model', metric, 'scaled']]

            print( f'*** {noise} - {scaler} - {dataset} - So luong model khac nhau : ' , selected_columns['Model'].nunique() ) 
            
            # Tìm baseline model có AUCROC < best_model AUCROC
            sorted_baseline = selected_columns.sort_values(by=metric, ascending=False)

             
            grouped = sorted_baseline.groupby("Model")

            baseline_rows = []
            for model_name, group in grouped:
                filtered = group[group[metric] < best_metric]
             
                if not filtered.empty:
                    best_row = filtered.loc[filtered[metric].idxmax()]
                    baseline_rows.append({
                        "Model": model_name,
                        col_name: best_row[metric]
                    })
                else:
                    baseline_rows.append({
                        "Model": model_name,
                        col_name: best_metric * 1.05  # gán 100 nếu không có model baseline nào thấp hơn
                    })
                    
            # Tạo dataframe kết quả

            report_df = pd.DataFrame([best_model_row.to_dict()] + baseline_rows )
                # report_df = pd.DataFrame(report_rows)
    
            # Ghi kết quả ra file
            report_path = f"Results/final/{metric}_report_noise_{noise}.csv"
            if os.path.exists(report_path):
                old_df = pd.read_csv(report_path)
                merged_df = pd.merge(old_df, report_df, on='Model', how='inner')
                merged_df.to_csv(report_path, index=False)
            else:
                report_df.to_csv(report_path, index=False)


*** 0 - RobustScaler - data_ToNIoT.csv - So luong model khac nhau :  16
*** 1 - RobustScaler - data_ToNIoT.csv - So luong model khac nhau :  16
*** 3 - RobustScaler - data_ToNIoT.csv - So luong model khac nhau :  16
*** 5 - RobustScaler - data_ToNIoT.csv - So luong model khac nhau :  16


In [45]:

# import pandas as pd
# import os

# # Đường dẫn các file dữ liệu
# baseline_file = "~/GMM-nfst/Results/Baseline_noise73.csv"
# model_files = {
#     # "data_BoTIoT.csv": "~/GMM-nfst/Results/OURMODEL/SCALERS/K_73_old_data_BoTIoT.csv0.csv",
#     # "data_CICIoT2023.csv": "~/GMM-nfst/Results/OURMODEL/SCALERS/K_73_old_data_CICIoT2023.csv0.csv",
#     # "data_N_BaIoT.csv" : "~/GMM-nfst/Results/OURMODEL/SCALERS/K_73_old_data_N_BaIoT.csv0.csv",
#     "data_ToNIoT.csv": "~/GMM-nfst/Results/OURMODEL/SCALERS/K_73_old_data_ToNIoT.csv0.csv"
# }

# scaler_names = ['MinMaxScaler', 'Normalizer', 'StandardScaler', 'QuantileTransformer', 'RobustScaler']
# metric_names = ['AUCROC'] # , 'AUCPR', 'Accuracy', 'MCC', 'F1 Score']   
# # Đọc dữ liệu baseline
# baseline_df = pd.read_csv(os.path.expanduser(baseline_file))

# # Lặp qua từng mức noise
# for noise in [0, 1, 3, 5]: 
#     baseline_noise = baseline_df[baseline_df["noise_percentage"] == noise]
#     baseline_noise = baseline_noise[baseline_noise['Model'] != 'LUNAR']

#     for dataset, model_file_path in model_files.items():
        

#         # Đọc dữ liệu mô hình của bạn
#         model_df = pd.read_csv(os.path.expanduser(model_file_path))
#         model_noise = model_df[model_df["noise_percentage"] == noise]

#         for metric in metric_names: 

#             best_metric = -1
        
#             # Tìm mô hình của bạn có AUCROC cao nhất
#             for scaler in scaler_names:
#                 model_scaler = model_noise[model_noise["scaler"] == scaler]
#                 for _, row in model_scaler.iterrows():
#                     if row[metric] > best_metric:
#                         best_metric = row[metric]
    
#             best_model_row = None
            
#             # if best_model_row is None:
#             #     best_model_row = pd.Series({'Model': 'ourmodel', dataset: 0})
#             # else:

#             col_name = f"{dataset}_{metric}"
#             best_model_row = pd.Series({'Model': 'ourmodel', col_name: best_metric})

            
#             # Lọc baseline theo dataset
#             dataset_baseline = baseline_noise[baseline_noise["Dataset"] == dataset]
#             selected_columns = dataset_baseline[['Model', metric, 'scaled']]

#             print( f'*** {noise} - {scaler} - {dataset} - So luong model khac nhau : ' , selected_columns['Model'].nunique() ) 
            
#             # Tìm baseline model có AUCROC < best_model AUCROC
#             sorted_baseline = selected_columns.sort_values(by=metric, ascending=False)

#             best_baseline_rows = [] 
#             best_loss = 1000  
            
#                 scaler_baseline = sorted_baseline[sorted_baseline["scaled"] == scaler] 
                
#                 grouped = scaler_baseline.groupby("Model")
    
#                 print( f' {noise} - {scaler} - {dataset} - So luong model khac nhau : ' ,  scaler_baseline['Model'].nunique() ) 
#                 baseline_rows = []
#                 for model_name, group in grouped:
#                     filtered = group[group[metric] < best_metric]
                 
#                     if not filtered.empty:
#                         best_row = filtered.loc[filtered[metric].idxmax()]
#                         baseline_rows.append({
#                             "Model": model_name,
#                             col_name: best_row[metric]
#                         })
#                     else:
#                         baseline_rows.append({
#                             "Model": model_name,
#                             col_name: best_metric * 1.05  # gán 100 nếu không có model baseline nào thấp hơn
#                         })
                        
#                 # Tạo dataframe kết quả
    
#                 size_tmp = sum(1 for row in baseline_rows if row[col_name] > best_metric)
                
#                 baseline_rows.append({'Model': 'Thua bao nhieu', col_name: size_tmp})

#                 if ( size_tmp < best_loss ): 
#                     best_loss = size_tmp 
#                     best_baseline_rows = baseline_rows.copy() 
            
#             report_df = pd.DataFrame([best_model_row.to_dict()] + best_baseline_rows )
#                 # report_df = pd.DataFrame(report_rows)
    
#             # Ghi kết quả ra file
#             report_path = f"Results/final/{metric}_report_noise_{noise}.csv"
#             if os.path.exists(report_path):
#                 old_df = pd.read_csv(report_path)
#                 merged_df = pd.merge(old_df, report_df, on='Model', how='inner')
#                 merged_df.to_csv(report_path, index=False)
#             else:
#                 report_df.to_csv(report_path, index=False)

*** 0 - RobustScaler - data_ToNIoT.csv - So luong model khac nhau :  16
 0 - MinMaxScaler - data_ToNIoT.csv - So luong model khac nhau :  16
 0 - Normalizer - data_ToNIoT.csv - So luong model khac nhau :  16
 0 - StandardScaler - data_ToNIoT.csv - So luong model khac nhau :  16
 0 - QuantileTransformer - data_ToNIoT.csv - So luong model khac nhau :  16
 0 - RobustScaler - data_ToNIoT.csv - So luong model khac nhau :  16
*** 1 - RobustScaler - data_ToNIoT.csv - So luong model khac nhau :  16
 1 - MinMaxScaler - data_ToNIoT.csv - So luong model khac nhau :  16
 1 - Normalizer - data_ToNIoT.csv - So luong model khac nhau :  16
 1 - StandardScaler - data_ToNIoT.csv - So luong model khac nhau :  16
 1 - QuantileTransformer - data_ToNIoT.csv - So luong model khac nhau :  16
 1 - RobustScaler - data_ToNIoT.csv - So luong model khac nhau :  16
*** 3 - RobustScaler - data_ToNIoT.csv - So luong model khac nhau :  16
 3 - MinMaxScaler - data_ToNIoT.csv - So luong model khac nhau :  16
 3 - Normal

In [8]:

        # report_df = pd.concat([best_model_df, sorted_baseline], ignore_index=True).fillna(0) 
        # # report_df.rename(columns={'AUCPR': dataset }, inplace=True)
        # report_df.columns.values[1] = dataset 
        # report_path = f"Results/final_report_noise_{noise}.csv"
    
        # if os.path.exists(report_path):
        #     # Nếu tồn tại thì merge vào file cũ
        #     old_df = pd.read_csv(report_path)

        #     print( old_df ) 
        #     print( report_df ) 
        #     merged_df = pd.merge(old_df, report_df, on='Model', how='inner')
            
        #     merged_df.to_csv(report_path, index=False)
        # else:
        #     # Tạo file mới
        #     report_df.to_csv(report_path, index=False)

# #Đường dẫn các file dữ liệu
# baseline_file = "~/GMM-nfst/Results/Baseline_noise73.csv"
# # model_files = {
# #     "data_BoTIoT.csv": "~/GMM-nfst/Results/OURMODEL/SCALERS/5%_data_BoTIoT.csv_100k_oldlearn0.csv",
# #     "data_CICIoT2023.csv": "~/GMM-nfst/Results/OURMODEL/SCALERS/5%_data_CICIoT2023.csv_100k_oldlearn0.csv",
# #     "data_N_BaIoT.csv" : "~/GMM-nfst/Results/OURMODEL/SCALERS/5%_data_N_BaIoT.csv_100k_oldlearn0.csv",
# # }

# model_files = {
#     "data_BoTIoT.csv": "~/GMM-nfst/Results/OURMODEL/SCALERS/K_73_old_data_BoTIoT.csv0.csv",
#     "data_CICIoT2023.csv": "~/GMM-nfst/Results/OURMODEL/SCALERS/K_73_old_data_CICIoT2023.csv0.csv",
#     "data_N_BaIoT.csv" : "~/GMM-nfst/Results/OURMODEL/SCALERS/K_73_old_data_N_BaIoT.csv0.csv",
#     "data_ToNIoT.csv": "~/GMM-nfst/Results/OURMODEL/SCALERS/73_old_data_ToNIoT.csv0.csv"
# }

# # ['N_BaIoT_dataloader.csv', 'data_CICIoT2023.csv','data_BoTIoT.csv']
# # Đọc dữ liệu baseline
# baseline_df = pd.read_csv(baseline_file)

# # Tìm giá trị AUC và AUCROC cao nhất trong baseline
# baseline_results = {}
# count = 0 

# # scaler_names = ['MinMaxScaler','QuantileTransformer', 'StandardScaler','QuantileTransformer']
# scaler_names = ['MinMaxScaler','Normalizer', 'StandardScaler','QuantileTransformer', 'RobustScaler' ]


# for noise in [0, 1,3,5] : 
    
#     baseline_noise = baseline_df[baseline_df["noise_percentage"] == noise]
#     baseline_noise = baseline_noise[baseline_noise['Model'] != 'LUNAR']
    
#     for dataset in model_files.keys():

#         '''
#         '''
#         if ( dataset == "data_BoTIoT.csv" ): 
#             dataset_baseline_noise = baseline_noise[baseline_noise['scaled'] == 'QuantileTransformer']
#         elif ( dataset == "data_CICIoT2023.csv" ) :
#             dataset_baseline_noise = baseline_noise[baseline_noise['scaled'] == 'MinMaxScaler']
#         else:
#             dataset_baseline_noise = baseline_noise[baseline_noise['scaled'] == 'MinMaxScaler']
        
#         best_model = None
#         best_loss = float('inf')
#         for scaler in scaler_names: 
            
#             dataset_baseline = dataset_baseline_noise[dataset_baseline_noise["Dataset"] == dataset]
#             max_auc = dataset_baseline["AUCPR"]
#             max_aucroc = dataset_baseline["AUCROC"]
#             model_name = dataset_baseline['Model'] 


#             selected_columns = dataset_baseline[['Model', 'AUCPR', 'AUCROC', 'scaled']]
                             
#             sorted_baseline = selected_columns.sort_values(by=['AUCROC'], ascending=[False])

            
#             model_df = pd.read_csv(model_files[dataset])
#             model_noise = model_df[model_df["noise_percentage"] == noise]
#             model_scaler = model_noise[model_noise["scaler"] == scaler] 
#             model_columns = model_scaler[['scaler', 'nCluster', 'AUCPR', 'AUCROC']]


           

#             for _, row in model_columns.iterrows():
#                 if best_model is None or row['AUCROC'] > best_model['AUCROC']:
#                     best_model = row
            
#         print( "*" * 50 , "\n", best_loss ) 
#         if best_model is None:
#             best_model = ["ourmodel", 0]  # Giá trị mặc định nếu không tìm thấy mô hình tốt nhất
#         else:
#             best_model = ["ourmodel"] + [best_model[3]] 

#         sorted_baseline = sorted_baseline[["Model", "AUCROC"]] 

#         best_model_df = pd.DataFrame([best_model], columns = ["Model", "AUCROC"] )

        
#         report_df = pd.concat([best_model_df, sorted_baseline], ignore_index=True).fillna(0) 
#         # report_df.rename(columns={'AUCPR': dataset }, inplace=True)
#         report_df.columns.values[1] = dataset 
#         report_path = f"Results/final_report_noise_{noise}.csv"
    
#         if os.path.exists(report_path):
#             # Nếu tồn tại thì merge vào file cũ
#             old_df = pd.read_csv(report_path)

#             print( old_df ) 
#             print( report_df ) 
#             merged_df = pd.merge(old_df, report_df, on='Model', how='inner')
            
#             merged_df.to_csv(report_path, index=False)
#         else:
#             # Tạo file mới
#             report_df.to_csv(report_path, index=False)
            